# 实战 Kaggle 比赛：图像分类 (CIFAR-10)

In [1]:
import torch
from d2l import torch as d2l
import collections
import math
import os
import shutil  # shutil库用于文件和文件夹的高级操作，例如复制、移动、删除等
import pandas as pd
import torch
import torchvision
from torch import nn

In [2]:
d2l.DATA_HUB['cifar10_tiny'] = (d2l.DATA_URL + 'kaggle_cifar10_tiny.zip', '2068874e4b9a9f0fb07ebe0ad2b29754449ccacd')

print(d2l.DATA_HUB)
demo = True

if demo:
    data_dir = d2l.download_extract('cifar10_tiny')
else:
    data_dir = "../data/cifar-10/"

{'airfoil': ('http://d2l-data.s3-accelerate.amazonaws.com/airfoil_self_noise.dat', '76e5be1548fd8222e5074cf0faae75edff8cf93f'), 'hotdog': ('http://d2l-data.s3-accelerate.amazonaws.com/hotdog.zip', 'fba480ffa8aa7e0febbb511d181409f899b9baa5'), 'banana-detection': ('http://d2l-data.s3-accelerate.amazonaws.com/banana-detection.zip', '5de26c8fce5ccdea9f91267273464dc968d20d72'), 'voc2012': ('http://d2l-data.s3-accelerate.amazonaws.com/VOCtrainval_11-May-2012.tar', '4e443f8a2eca6b1dac8a6c57641b67dd40621a49'), 'cifar10_tiny': ('http://d2l-data.s3-accelerate.amazonaws.com/kaggle_cifar10_tiny.zip', '2068874e4b9a9f0fb07ebe0ad2b29754449ccacd'), 'dog_tiny': ('http://d2l-data.s3-accelerate.amazonaws.com/kaggle_dog_tiny.zip', '0cb91d09b814ecdc07b50f31f8dcad3e81d6a86d'), 'ptb': ('http://d2l-data.s3-accelerate.amazonaws.com/ptb.zip', '319d85e578af0cdc590547f26231e4e31cdf1e42'), 'glove.6b.50d': ('http://d2l-data.s3-accelerate.amazonaws.com/glove.6B.50d.zip', '0b8703943ccdb6eb788e6f091b8946e82231bc4d'), 

In [3]:
c =dict({
    "a":"b"
})
print(c)

{'a': 'b'}


In [5]:
def read_csv_labels(fname):
    with open(fname, 'r') as f:
        lines = f.readlines()[1:]
    tokens = [l.rstrip().split(',') for l in lines]
    return dict(((name, label) for name,label in tokens))

labels = read_csv_labels(os.path.join(data_dir, 'trainLabels.csv'))
print('# 训练样本 :', len(labels))
print('# 类别 :', len(set(labels.values())))
for i, (k, v) in enumerate(labels.items()):
    if i < 10:
        print(f'{k}: {v}')
    else:
        break


# 训练样本 : 1000
# 类别 : 10
1: frog
2: truck
3: truck
4: deer
5: automobile
6: automobile
7: bird
8: horse
9: ship
10: cat


In [6]:
def copyfile(filename, target_dir):
    os.makedirs(target_dir, exist_ok=True)
    # shutil.copy(filename, target_dir)  # 只复制内容和权限信息，不包括mtime/atime等元数据
    shutil.copy2(filename, target_dir)   # 复制内容、权限和大多数元数据（包括时间戳）

def reorg_train_valid(data_dir, labels, valid_ratio):
    n = collections.Counter(labels.values()).most_common()[-1][1]
    n_valid_per_label = max(1, math.floor(n*valid_ratio))
    label_count = {}
    for train_file in os.listdir(os.path.join(data_dir, 'train')):
        label = labels[train_file.split('.')[0]]
        fname = os.path.join(data_dir, 'train', train_file)
        copyfile(fname, os.path.join(data_dir, 'train_valid_test', 'train_valid', label))
        if label not in label_count or label_count[label] < n_valid_per_label:
            copyfile(fname, os.path.join(data_dir, 'train_valid_test', 'valid', label))
            label_count[label] = label_count.get(label, 0) +1
        else:
            copyfile(fname, os.path.join(data_dir, 'train_valid_test', 'train', label))
    
    return n_valid_per_label


In [7]:
def reorg_test(data_dir):
    for test_file in os.listdir(os.path.join(data_dir, 'test')):
        copyfile(
            os.path.join(data_dir, 'test', test_file),
            os.path.join(data_dir, 'train_valid_test', 'test', 'unknown')
        )

In [8]:
def reorg_cifar10_data(data_dir, valid_ratio):
    labels = read_csv_labels(os.path.join(data_dir, 'trainLabels.csv'))
    reorg_train_valid(data_dir, labels, valid_ratio)
    reorg_test(data_dir)

In [17]:
batch_size = 32 if demo else 128
valid_ratio = 0.1
reorg_cifar10_data(data_dir, valid_ratio)


In [10]:
print(data_dir)

../data/kaggle_cifar10_tiny


In [14]:
transform_train = torchvision.transforms.Compose([
    # 首先，我们将图片的较短边缩放到40像素（长边等比例缩放），这样为后面的随机裁剪提供更高分辨率的图像基础。
    torchvision.transforms.Resize(40),
    # 然后，使用RandomResizedCrop随机裁剪一块32x32的区域。裁剪区域的面积占原图的比例为0.6到1之间，裁剪框的宽高比固定为1（即正方形），最后将该区域缩放（若有必要）到32x32。
    torchvision.transforms.RandomResizedCrop(32, scale=(0.6, 1), ratio=(1,1)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
                                     [0.2023, 0.1994, 0.2010])])

transorm_test = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
                                     [0.2023, 0.1994, 0.2010])
])

In [15]:
train_ds, train_valid_ds = [
    torchvision.datasets.ImageFolder(
        os.path.join(data_dir, 'train_valid_test', fold_name),
        transform=transform_train, 
    ) for fold_name in ['train', 'train_valid']
]

valid_ds, test_ds = [
    torchvision.datasets.ImageFolder(
        os.path.join(data_dir, 'train_valid_test', fold_name),
        transform=transorm_test,
    ) for fold_name in ['valid', 'test']
]

In [18]:
train_iter, test_iter = [torch.utils.data.DataLoader(
    dataset, batch_size, shuffle=True, drop_last = True
) for dataset in (train_ds, train_valid_ds)]

valid_iter= torch.utils.data.DataLoader(
    valid_ds, batch_size, shuffle=True, drop_last=True
)

test_iter = torch.utils.data.DataLoader(
    test_ds, batch_size, shuffle=True, drop_last=True
)

In [19]:
def get_net():
    num_classes = 10
    net = d2l.resnet18(num_classes, 3)
    return net

loss = nn.CrossEntropyLoss(reduction='none')

In [20]:
def train(net, train_iter, valid_iter, num_epochs, lr, wd, devices, lr_perios, lr_decay):
    trainer = torch.optim.SGD(net.parameters(),lr, momentum=0.9, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.StepLR(trainer, lr_perios, lr_decay)
    num_batches, timer = len(train_iter), d2l.Timer()
    legend = ['train loss', 'train acc']
    if valid_iter is not None:
        legend.append('valid acc')
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs],
                            legend=legend)
    
    net = nn.DataParallel(net, devices).to(devices[0])
    for epoch in range(num_epochs):
        net.train()
        metric = d2l.Accumulator(3)
        for i, (features, labels) in enumerate(train_iter):
            timer.start()
            l, acc = d2l.train_batch_ch13(net, features, labels, loss, scheduler, devices)
            metric.add(l, acc, labels.shape[0])
            timer.stop()
            if (i+1) % (num_batches // 5) ==0 or i == num_batches -1:
                animator.add(epoch + (i+1)/num_batches, (metric[0]/metric[2], metric[1]/metric[2], None))
        
        if valid_iter is not None:
            valid_acc = d2l.evaluate_accuracy_gpu(net, valid_iter, devices)
            animator.add(epoch +1, (None, None, valid_acc))
        scheduler.step()
    measures = (f'train loss {metric[0] / metric[2]:.3f}, '
                f'train acc {metric[1] / metric[2]:.3f}')
    if valid_iter is not None:
        measures += f", valid acc {valid_acc:.3f}"
    print(measures + f'\n{metric[2] * num_epochs / timer.sum():.1f}'
          f' examples/sec on {str(devices)}')

In [ ]:
devices, num_epochs, lr, wd = d2l.try_all_gpus(), 10, 2e-4, 5e-4
lr_perios, lr_decay, net = 4, 0.9, get_net()
train(net, train_iter, valid_iter, num_epochs, lr, wd, devices, lr_perios, lr_decay)

In [ ]:
net, preds = get_net(), []
train(net, train_iter, None, num_epochs, lr, wd, devices, lr_perios, lr_decay)

for X,_ in test_iter:
    y_hat = net(X).to(devices[0])
    preds.extend(y_hat.argmax(dim=1).type(torch.float32).cpu().numpy())
sorted_ids = list(range(1, len(test_ds) +1))
sorted_ids.sort(key = lambda x: str(x))
df = pd.DataFrame({'id': sorted_ids, 'label': preds})
# classes是一个保存所有类别名称的列表，比如['airplane', 'automobile', ...]，它的索引对应模型输出的类别编号。用train_valid_ds.classes[x]可以把预测的类别编号x转换为类别的实际名称。
df['label'] = df['label'].apply(lambda x: train_valid_ds.classes[int(x)])
df.to_csv('submission.csv', index=False)